## Advent of Code 2023 Day 2 🟥 🟩 🟦 

This is my first time trying out the [FParsec](https://github.com/stephan-tolksdorf/fparsec) library.

For me the most painful/tedious part of Advent of Code is parsing the input data with a collection of `String.Split`, `String.Trim` and various array operations.

FParsec is far nicer to work with and I think I'll be using it from now on.

In [36]:
#r "nuget:FParsec"

open FParsec

let loadFile fname = System.IO.File.ReadAllLines(fname)

let sample = "day2.sample.txt"
let data = "day2.data.txt"

Installed Packages FParsec, 1.1.1

`parseRow` will return an `int * List<List<int * string>>`, representing `(gameId, [[(count, color)...] ... ])`, or throw an error if the data is bad. (In actual fact, as the data is most likely good, it would indicate that my parser is bad).

The parser is built up from smaller parser combinators, which each parsing one thing, or combining other parsers in some way. E.g.:

- a string
- an int
- a tuple made from running other parsers in sequence
- a list parser, made from specifying a seperator and the inner parser
- etc. (there's far more that I haven't needed or learned about yet)

This method of atomically building up a parser from primitives seems to work really well.

In [37]:
let str = pstring
let gameNumberParser = str "Game " >>. pint32 .>> str ": " // parse the Game ID
let colorNameParser = str "red" <|> str "blue" <|> str "green" // parse one of strings red|blue|green. Anything else will be a parse failure.
let colorCountParser = tuple2 (pint32 .>> str " ") colorNameParser // a 2-tuple parser combinator of a count parser and a color parser
let colorCountListParser = sepBy colorCountParser (str ", ") // apply the colorCountParser to a comma seperated list. This is one "handful"
let drawListParser = sepBy colorCountListParser (str "; ") // apply the above parser to a semicolon list. These are the handfuls for one "game"
let gameParser = tuple2 gameNumberParser drawListParser // combine the game number parser, and the "handfuls" parser into a 2-tuple parser.

let parseRow (s: string) =
    match run gameParser s with
    | Success (result, _, _) -> result
    | Failure (message, _, _) -> failwith message

let testRow = "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"
let parsedRow = parseRow testRow

printfn $"Test game parser: {parsedRow}"

Test game parser: (5, [[(6, red); (1, blue); (3, green)]; [(2, blue); (1, red); (2, green)]])


Here's an example of the parser failing. It's pretty neat actually...

In [38]:
let nicerParseRow (s: string) =
    match run gameParser s with
    | Success (result, _, _) -> failwith "This wasn't meant to succeed..."
    | Failure (message, _, _) -> message

let invalidTestRow = "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 pink, 2 green"
let parsedRowErrorMessage = nicerParseRow invalidTestRow

display "NOTE: the following error was intentional"
printfn $"Test game parser on bad data: {parsedRowErrorMessage}"

Test game parser on bad data: Error in Ln: 1 Col: 43
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 pink, 2 green
                                          ^
Expecting: 'blue', 'green' or 'red'



In [39]:
type CubeCount = private CubeCount of int array

module CubeCount =
    let create r g b = CubeCount [| r ; g ; b |]

    let value cubeCount = 
        match cubeCount with CubeCount arr -> arr

    let couldBeDrawnFrom bagCount drawCount =
        (value bagCount, value drawCount)
        ||> Array.forall2 (>=)

    let private countForColorFromTuples color xs =
        xs
        |> List.tryFind (fun (_, c) -> c = color)
        |> Option.map fst
        |> Option.defaultValue 0

    let fromTupleList (xs : (int * string) list) =
        CubeCount [|
            xs |> countForColorFromTuples "red"
            xs |> countForColorFromTuples "green"
            xs |> countForColorFromTuples "blue"
            |]

    let maximalArray<'a> =
        Array.map2 max

    let maximalDraw (draws: CubeCount list) =
        draws
        |> List.reduce (fun (CubeCount a) (CubeCount b) ->
            CubeCount (maximalArray a b))

    let product cubeCount =
        match cubeCount with
        | CubeCount [| r; g; b |] -> r * g * b


printfn $"""{CubeCount.fromTupleList [ (3, "blue"); (56, "red") ]}"""



CubeCount [|56; 0; 3|]


In [40]:
let bagConfig = CubeCount.create 12 13 14

display "Part One Answer:"

loadFile data
|> Array.map parseRow
|> Array.map (fun (g, c) -> (g, c |> List.map CubeCount.fromTupleList))
|> Array.filter (fun (g, draws) ->
    draws |> List.forall (fun draw -> draw |> CubeCount.couldBeDrawnFrom bagConfig)) 
|> Array.sumBy fst



Part One Answer:

2285

In [41]:
display "Part Two Answer:"

loadFile data
|> Array.map (parseRow >> snd >> List.map CubeCount.fromTupleList )
|> Array.map CubeCount.maximalDraw
|> Array.map CubeCount.product
|> Array.sum





Part Two Answer:

77021